In [ ]:
import numpy as np
import cv2
from ffpyplayer.player import MediaPlayer
import skvideo.io
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip
import pytesseract      
from PIL import Image as im
import pyttsx3
import spacy
import os
import re
import srt

In [ ]:
for video_sirasi in range(16):
    print(str(video_sirasi+1) + ". Video İşleniyor..")
    #Videoyu oku#


    try:
        path = r"D:\Burak\Desktop\Proje\Ham\\" + str(video_sirasi) + ".mp4"
        videodata = skvideo.io.vread(path, as_grey=True)
    
    except:
        continue
    #Videoyu oku#
    
    #Variables#
    beyaz_initial = 0
    beyaz = 0
    alt = 615
    ust = 565
    text_change = [0]
    length = len(videodata)
    sinir = 255
    frame = 10    
    b = videodata[0][ust:alt]
    length_b = len(b)
    #Variables#

    #Cümle değişimlerini algıla#
    for i in range (len(b)):
            for j in range (len(b[i])):
                #if b[i][j][0] > sinir and b[i][j][1] > sinir and b[i][j][2] > sinir:
                if b[i][j][0]== sinir:
                    beyaz_initial +=1
                else:
                    pass

    for k in range (len(videodata)):
        if k % frame == 0:
            b = videodata[k][ust:alt]
            beyaz = 0
            for i in range (len(b)):
                for j in range (len(b[i])):
                    #if b[i][j][0] > sinir and b[i][j][1] > sinir and b[i][j][2] > sinir:
                    if b[i][j][0]== sinir:
                        beyaz +=1
                    else:
                        pass
            if beyaz_initial*0.98 <= beyaz <= beyaz_initial*1.1:
                pass
            else:
                if text_change[-1] + 40 > k:
                    pass
                else:
                    beyaz_initial = beyaz
                    text_change.append(k)

        else:
            pass
    #Cümle değişimlerini algıla#

    #Variables#
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    t1 = 0
    #Variables#

    #Videoyu kes#
    for i in range (len(text_change)):
        print (str(i+1)+"/"+str(len(text_change)))
        name = r"D:\Burak\Desktop\Proje\Cumle\\" + str(video_sirasi)+ "." +str(i) + ".mp4"
        if i == len(text_change)-1:
            t1 = text_change[-1]/fps 
            t2 = len(videodata)/fps
            new = VideoFileClip(path).subclip(t1-0.2, t2)
            new.write_videofile(name, threads=10, fps=fps, verbose=False, logger=None)
        elif i == 0:
            t1 = 0
            t2 =  text_change[1]/fps
            new = VideoFileClip(path).subclip(t1, t2)
            new.write_videofile(name, threads=10, fps=fps, verbose=False, logger=None)
        else:
            t1 = text_change[i]/fps
            t2 = text_change[i+1]/fps
            new = VideoFileClip(path).subclip(t1-0.2, t2)
            new.write_videofile(name, threads=10, fps=fps, verbose=False, logger=None)   
    #Videoyu kes#

    #Resimleri Oku#
    results = []
    sira = 0
    for i in range (len(text_change)):
        if i == len(text_change)-1:
            sira = text_change[i]
        else:
            sira = int((text_change[i] + text_change[i+1])/2)
        np_img = np.squeeze(videodata[sira][ust:alt], axis=2)
        img = im.fromarray(np_img)
        result = pytesseract.image_to_string(img, lang='deu_frak')
        results.append(result)
    #Resimleri Oku#

    #Cümleleri yazdır#
    with open("D:\Burak\Desktop\Proje\Text\\"+str(video_sirasi)+".txt", 'w') as f:
        cumle_sirasi = 0
        for item in results:
            f.write(str(cumle_sirasi) +"-"+ "%s\n" % item)
            cumle_sirasi+=1
    #Cümleleri yazdır#
    
    #Dataları temizle
    del cap
    del fps
    del t1
    del beyaz_initial
    del beyaz
    del text_change
    del length
    del length_b
    del b
    del videodata
    #Dataları temizle

1. Video İşleniyor..


In [2]:
nlp = spacy.load('en_core_web_sm')

for video_sirasi in range(16):
    print(str(video_sirasi+1) + ". Video İşleniyor..")
    
    #Videoyu oku#
    try:
        path = r"D:\Burak\Desktop\Proje\Ham\\" + str(video_sirasi) + ".mp4"
        videodata = skvideo.io.vread(path, as_grey=True)
    
    except:
        continue
    #Videoyu oku#
    
    #Variables#
    alt = 615
    ust = 565
    frame = 10
    text_change = []
    results = []
    length = len(videodata)    
    #Variables#

    #Cümle değişimlerini algıla#
    for i in range (length):
        if i % frame == 0:
            print("Cümleler Okunuyor..." + str(i) + "/" + str(length))
            np_img = np.squeeze(videodata[i][ust:alt], axis=2)
            img = im.fromarray(np_img)
            result = pytesseract.image_to_string(img, lang='deu_frak')
            if len(results) == 0:
                results.append(result)
                text_change.append(i)
            elif result not in results:
                doc1 = nlp(result)
                doc2 = nlp(results[-1])
                if doc1.similarity(doc2) < 0.8:
                    results.append(result)
                    text_change.append(i)
                    print(result)
                else:
                    pass
            else:
                pass
    #Cümle değişimlerini algıla#

    #Variables#
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    t1 = 0
    #Variables#

    #Videoyu kes#
    for i in range (len(text_change)):
        print (str(i+1)+"/"+str(len(text_change)))
        name = r"D:\Burak\Desktop\Proje\Cumle\\" + str(video_sirasi)+ "." +str(i) + ".mp4"
        if i == len(text_change)-1:
            t1 = text_change[-1]/fps 
            t2 = len(videodata)/fps
            new = VideoFileClip(path).subclip(t1-0.2, t2)
            new.write_videofile(name, threads=4, fps=fps, verbose=False, logger=None)
        elif i == 0:
            t1 = 0
            t2 =  text_change[1]/fps
            new = VideoFileClip(path).subclip(t1, t2)
            new.write_videofile(name, threads=4, fps=fps, verbose=False, logger=None)
        else:
            t1 = text_change[i]/fps
            t2 = text_change[i+1]/fps
            new = VideoFileClip(path).subclip(t1-0.2, t2)
            new.write_videofile(name, threads=4, fps=fps, verbose=False, logger=None)   
    #Videoyu kes#

    #Cümleleri yazdır#
    with open("D:\Burak\Desktop\Proje\Text\\"+str(video_sirasi)+".txt", 'w') as f:
        cumle_sirasi = 0
        for item in results:
            f.write(str(cumle_sirasi) +"-"+ "%s\n" % item)
            cumle_sirasi+=1
    #Cümleleri yazdır#
    
    #Dataları temizle
    del cap
    del fps
    del t1
    del text_change
    del length
    del videodata
    #Dataları temizle

1. Video İşleniyor..
Cümleler Okunuyor...0/14132
Cümleler Okunuyor...10/14132
Cümleler Okunuyor...20/14132
Cümleler Okunuyor...30/14132
Cümleler Okunuyor...40/14132
In unserem heutigen Video möchte ich mit euch darüber sprechen,


Cümleler Okunuyor...50/14132
Cümleler Okunuyor...60/14132
Cümleler Okunuyor...70/14132
Cümleler Okunuyor...80/14132
Cümleler Okunuyor...90/14132
Cümleler Okunuyor...100/14132
Cümleler Okunuyor...110/14132
Cümleler Okunuyor...120/14132
Cümleler Okunuyor...130/14132
Cümleler Okunuyor...140/14132
wie man im Deutschen Sätze verneint.


Cümleler Okunuyor...150/14132
Cümleler Okunuyor...160/14132
Cümleler Okunuyor...170/14132
Cümleler Okunuyor...180/14132
Cümleler Okunuyor...190/14132
Cümleler Okunuyor...200/14132
Cümleler Okunuyor...210/14132
Cümleler Okunuyor...220/14132
Tatsächlich gibt es im Deutschen zwei Methoden, eine Aussage zu verneinen-


Cümleler Okunuyor...230/14132
Cümleler Okunuyor...240/14132
Cümleler Okunuyor...250/14132
Cümleler Okunuyor...260/1

Cümleler Okunuyor...2310/14132
Cümleler Okunuyor...2320/14132
und dieses Nomen hat einen unbestimmten Artikel. "Ein" dummer Mensch.


Cümleler Okunuyor...2330/14132
Cümleler Okunuyor...2340/14132
Cümleler Okunuyor...2350/14132
Cümleler Okunuyor...2360/14132
Cümleler Okunuyor...2370/14132
Cümleler Okunuyor...2380/14132
Cümleler Okunuyor...2390/14132
Cümleler Okunuyor...2400/14132
Cümleler Okunuyor...2410/14132
Cümleler Okunuyor...2420/14132
Cümleler Okunuyor...2430/14132
Cümleler Okunuyor...2440/14132
Cümleler Okunuyor...2450/14132
Cümleler Okunuyor...2460/14132
Und deshalb benutzt man hier "kein" zur Verneinung.


Cümleler Okunuyor...2470/14132
Cümleler Okunuyor...2480/14132
Cümleler Okunuyor...2490/14132
Cümleler Okunuyor...2500/14132
Cümleler Okunuyor...2510/14132
Cümleler Okunuyor...2520/14132
Cümleler Okunuyor...2530/14132
Cümleler Okunuyor...2540/14132
Cümleler Okunuyor...2550/14132
Cümleler Okunuyor...2560/14132
Cümleler Okunuyor...2570/14132
Cümleler Okunuyor...2580/14132
Cüml

Cümleler Okunuyor...4450/14132
Cümleler Okunuyor...4460/14132
Cümleler Okunuyor...4470/14132
Cümleler Okunuyor...4480/14132
Im Englischen ist die Satzkonstruktion hier sehr ähnlich


Cümleler Okunuyor...4490/14132
Cümleler Okunuyor...4500/14132
Cümleler Okunuyor...4510/14132
Cümleler Okunuyor...4520/14132
Cümleler Okunuyor...4530/14132
Cümleler Okunuyor...4540/14132
Cümleler Okunuyor...4550/14132
Cümleler Okunuyor...4560/14132
oder die Übersetzung sehr ähnlich. "kein Alkohol" = "no alcohol".


Cümleler Okunuyor...4570/14132
Cümleler Okunuyor...4580/14132
Cümleler Okunuyor...4590/14132
Cümleler Okunuyor...4600/14132
Cümleler Okunuyor...4610/14132
Cümleler Okunuyor...4620/14132
Cümleler Okunuyor...4630/14132
Cümleler Okunuyor...4640/14132
Cümleler Okunuyor...4650/14132
Cümleler Okunuyor...4660/14132
Cümleler Okunuyor...4670/14132
Cümleler Okunuyor...4680/14132
Cümleler Okunuyor...4690/14132
Sobald wir jedoch einen vollständigen Satz haben,


Cümleler Okunuyor...4700/14132
Cümleler Oku

Cümleler Okunuyor...6710/14132
Cümleler Okunuyor...6720/14132
Cümleler Okunuyor...6730/14132
Cümleler Okunuyor...6740/14132
Cümleler Okunuyor...6750/14132
Cümleler Okunuyor...6760/14132
Cümleler Okunuyor...6770/14132
Cümleler Okunuyor...6780/14132
Cümleler Okunuyor...6790/14132
Cümleler Okunuyor...6800/14132
Cümleler Okunuyor...6810/14132
Cümleler Okunuyor...6820/14132
Cümleler Okunuyor...6830/14132
Cümleler Okunuyor...6840/14132
Cümleler Okunuyor...6850/14132
Cümleler Okunuyor...6860/14132
Cümleler Okunuyor...6870/14132
Cümleler Okunuyor...6880/14132
Und dieses "nicht" und dessen Satzkonstruktion,


Cümleler Okunuyor...6890/14132
Cümleler Okunuyor...6900/14132
Cümleler Okunuyor...6910/14132
Cümleler Okunuyor...6920/14132
Cümleler Okunuyor...6930/14132
Cümleler Okunuyor...6940/14132
Cümleler Okunuyor...6950/14132
Cümleler Okunuyor...6960/14132
gucken wir uns jetzt genauer an.


Cümleler Okunuyor...6970/14132
Cümleler Okunuyor...6980/14132
Cümleler Okunuyor...6990/14132
Cümleler Okunu

Cümleler Okunuyor...8970/14132
Cümleler Okunuyor...8980/14132
Cümleler Okunuyor...8990/14132
Cümleler Okunuyor...9000/14132
Cümleler Okunuyor...9010/14132
Cümleler Okunuyor...9020/14132
Cümleler Okunuyor...9030/14132
und manchmal ändert Sich dadurch sogar die satzaussage leicht,


Cümleler Okunuyor...9040/14132
Cümleler Okunuyor...9050/14132
Cümleler Okunuyor...9060/14132
Cümleler Okunuyor...9070/14132
Cümleler Okunuyor...9080/14132
Cümleler Okunuyor...9090/14132
Cümleler Okunuyor...9100/14132
Cümleler Okunuyor...9110/14132
Cümleler Okunuyor...9120/14132
Cümleler Okunuyor...9130/14132
wie in diesem Beispiel.


Cümleler Okunuyor...9140/14132
Cümleler Okunuyor...9150/14132
Cümleler Okunuyor...9160/14132
Cümleler Okunuyor...9170/14132
Cümleler Okunuyor...9180/14132
Ich gehe heute nicht einkaufen.


Cümleler Okunuyor...9190/14132
Cümleler Okunuyor...9200/14132
Cümleler Okunuyor...9210/14132
Cümleler Okunuyor...9220/14132
Cümleler Okunuyor...9230/14132
Cümleler Okunuyor...9240/14132
Cüml

...

Cümleler Okunuyor...8150/17434
Cümleler Okunuyor...8160/17434
Cümleler Okunuyor...8170/17434
Cümleler Okunuyor...8180/17434
Cümleler Okunuyor...8190/17434
Cümleler Okunuyor...8200/17434
Cümleler Okunuyor...8210/17434
Cümleler Okunuyor...8220/17434
Cümleler Okunuyor...8230/17434
Ähm... — Wein trinken. - Ja, So ziemlich, genau.


Cümleler Okunuyor...8240/17434
Cümleler Okunuyor...8250/17434
Cümleler Okunuyor...8260/17434
Cümleler Okunuyor...8270/17434
Cümleler Okunuyor...8280/17434
Cümleler Okunuyor...8290/17434
Cümleler Okunuyor...8300/17434
Cümleler Okunuyor...8310/17434
Cümleler Okunuyor...8320/17434
Cümleler Okunuyor...8330/17434
Cümleler Okunuyor...8340/17434
Cümleler Okunuyor...8350/17434
Cümleler Okunuyor...8360/17434
Cümleler Okunuyor...8370/17434
Cümleler Okunuyor...8380/17434
Cümleler Okunuyor...8390/17434
Also wir haben, wir wollten eigentlich noch 'n bisschen malen


Cümleler Okunuyor...8400/17434
Cümleler Okunuyor...8410/17434
Cümleler Okunuyor...8420/17434
Cümleler Okunuy

Cümleler Okunuyor...10280/17434
Cümleler Okunuyor...10290/17434
ja, dann gucken wir noch 'ne... äh, einen Film


Cümleler Okunuyor...10300/17434
Cümleler Okunuyor...10310/17434
Cümleler Okunuyor...10320/17434
Cümleler Okunuyor...10330/17434
Cümleler Okunuyor...10340/17434
Cümleler Okunuyor...10350/17434
Cümleler Okunuyor...10360/17434
Cümleler Okunuyor...10370/17434
oder vielleicht machen wir noch mal was, irgendwas anderes.


Cümleler Okunuyor...10380/17434
Cümleler Okunuyor...10390/17434
Cümleler Okunuyor...10400/17434
Cümleler Okunuyor...10410/17434
Cümleler Okunuyor...10420/17434
Cümleler Okunuyor...10430/17434
Cümleler Okunuyor...10440/17434
Cümleler Okunuyor...10450/17434
Cümleler Okunuyor...10460/17434
Cümleler Okunuyor...10470/17434
Cümleler Okunuyor...10480/17434
Was habt ihr heute Abend vor?


Cümleler Okunuyor...10490/17434
Cümleler Okunuyor...10500/17434
Cümleler Okunuyor...10510/17434
Cümleler Okunuyor...10520/17434
Cümleler Okunuyor...10530/17434
Ähm, noch nichts groBa

Cümleler Okunuyor...12440/17434
Cümleler Okunuyor...12450/17434
Wenn jetzt corona nicht wär'? lVlmh...


Cümleler Okunuyor...12460/17434
Cümleler Okunuyor...12470/17434
Cümleler Okunuyor...12480/17434
wenn jetzt cordna nicht wär-? Mmh...


Cümleler Okunuyor...12490/17434
Cümleler Okunuyor...12500/17434
Cümleler Okunuyor...12510/17434
Cümleler Okunuyor...12520/17434
Cümleler Okunuyor...12530/17434
Wenn jetzt corona nicht wär'? Mmh...


Cümleler Okunuyor...12540/17434
Cümleler Okunuyor...12550/17434
Cümleler Okunuyor...12560/17434
Cümleler Okunuyor...12570/17434
Cümleler Okunuyor...12580/17434
Cümleler Okunuyor...12590/17434
Oh, ich würde, glaube ich, tatsächlich einfach


Cümleler Okunuyor...12600/17434
Cümleler Okunuyor...12610/17434
Cümleler Okunuyor...12620/17434
Cümleler Okunuyor...12630/17434
Cümleler Okunuyor...12640/17434
Cümleler Okunuyor...12650/17434
Cümleler Okunuyor...12660/17434
Cümleler Okunuyor...12670/17434
alle meine Freunde zusammen zu einem riesengrolzen Essen,



Cümleler Okunuyor...14570/17434
Cümleler Okunuyor...14580/17434
Cümleler Okunuyor...14590/17434
Cümleler Okunuyor...14600/17434
Cümleler Okunuyor...14610/17434
Cümleler Okunuyor...14620/17434
Cümleler Okunuyor...14630/17434
Das ist ein schöner Traum. — Ja, genau.


Cümleler Okunuyor...14640/17434
Cümleler Okunuyor...14650/17434
Cümleler Okunuyor...14660/17434
Cümleler Okunuyor...14670/17434
Cümleler Okunuyor...14680/17434
Cümleler Okunuyor...14690/17434
Cümleler Okunuyor...14700/17434
Cümleler Okunuyor...14710/17434
Wahrscheinlich das Gleiche.


Cümleler Okunuyor...14720/17434
Cümleler Okunuyor...14730/17434
Cümleler Okunuyor...14740/17434
Cümleler Okunuyor...14750/17434
Cümleler Okunuyor...14760/17434
Cümleler Okunuyor...14770/17434
Cümleler Okunuyor...14780/17434
Cümleler Okunuyor...14790/17434
Oder vielleicht würden wir in lne Bar gehen


Cümleler Okunuyor...14800/17434
Cümleler Okunuyor...14810/17434
Cümleler Okunuyor...14820/17434
Cümleler Okunuyor...14830/17434
und was trinken

Cümleler Okunuyor...16700/17434
Cümleler Okunuyor...16710/17434
Cümleler Okunuyor...16720/17434
Cümleler Okunuyor...16730/17434
Cümleler Okunuyor...16740/17434
Cümleler Okunuyor...16750/17434
Cümleler Okunuyor...16760/17434
Cümleler Okunuyor...16770/17434
und euer Deutsch, dann bekommt ihr nämlich


Cümleler Okunuyor...16780/17434
Cümleler Okunuyor...16790/17434
Cümleler Okunuyor...16800/17434
Cümleler Okunuyor...16810/17434
zu diesem Video und zu allen anderen Videos


Cümleler Okunuyor...16820/17434
Cümleler Okunuyor...16830/17434
Cümleler Okunuyor...16840/17434
Cümleler Okunuyor...16850/17434
Cümleler Okunuyor...16860/17434
Cümleler Okunuyor...16870/17434
Transkripte, Vokabeln und interaktive Ubungen.


Cümleler Okunuyor...16880/17434
Cümleler Okunuyor...16890/17434
Cümleler Okunuyor...16900/17434
Cümleler Okunuyor...16910/17434
Cümleler Okunuyor...16920/17434
Cümleler Okunuyor...16930/17434
Cümleler Okunuyor...16940/17434
Cümleler Okunuyor...16950/17434
Cümleler Okunuyor...16960

In [24]:
#Video temizleme#
for j in range (16):    
    txt = open(r"D:\Burak\Desktop\Proje\Text\\" + str(j)+ ".txt")
    txt_list = txt.readlines()
    elemanlar = []
    for i in range (len(txt_list)):
        DY = True
        sira = 1
        try:
            deneme = int(txt_list[i][0])
            while DY:
                try:
                    deneme2 = int(txt_list[i][sira])
                    deneme = deneme*10 + deneme2
                    sira += 1
                except:
                    DY = False
                    elemanlar.append(deneme)
                    break
        except:
            continue
    video_sil = True
    devam = 0
    sira = 0
    while video_sil:
        name = "D:\Burak\Desktop\Cumle\\" + str(j) + "." + str(sira) + ".mp4"
        if os.path.exists(name) and sira not in elemanlar:
            os.remove(name)
            devam = 0
        elif os.path.exists(name) and sira in elemanlar:
            devam = 0
            pass
        else:
            devam +=1
            pass
        if devam > 50:
            video_sil = False
            break
        sira += 1

In [122]:
#Video adı değiştirme
for j in range (16):    
    txt = open(r"D:\Burak\Desktop\Proje\Text\\" + str(j)+ ".txt")
    txt_list = txt.readlines()
    elemanlar = []
    for i in range (len(txt_list)):
        DY = True
        sira = 1
        try:
            deneme = int(txt_list[i][0])
            while DY:
                try:
                    deneme2 = int(txt_list[i][sira])
                    deneme = deneme*10 + deneme2
                    sira += 1
                except:
                    DY = False
                    elemanlar.append(deneme)
                    break
        except:
            continue
            
    while '\x0c\n' in txt_list:
        txt_list.remove('\x0c\n')
    
    for i in range (len(txt_list)):
        xxx = re.findall("[A-Za-z_.]*", txt_list[i])
        
        while "" in xxx:
            xxx.remove("")
        yyy = ' '.join(xxx)
        txt_list[i] = yyy
    for i in range (len(elemanlar)):
        try:
            name1 = "D:\Burak\Desktop\Cumle\\" + str(j) + "." + str(elemanlar[i]) + ".mp4"
            print(txt_list[i])
            name2 = "D:\Burak\Desktop\Cumle\\" + txt_list[i] + ".mp4"
            os.rename(name1 ,name2)
        except:
            try:
                name1 = "D:\Burak\Desktop\Cumle\\" + str(j) + "." + str(elemanlar[i]) + ".mp4"
                print(txt_list[i])
                name2 = "D:\Burak\Desktop\Cumle\\" + txt_list[i]+ str(i) + ".mp4"
                os.rename(name1 ,name2)
            except:
                pass
            pass

Hallo Leute
Hallo Leute
In unserem heutigen Video m chte ich mit euch dar ber sprechen
In unserem heutigen Video m chte ich mit euch dar ber sprechen
wie man im Deutschen S tze verneint.
wie man im Deutschen S tze verneint.
Tats chlich gibt es im Deutschen zwei Methoden eine Aussage zu verneinen
Tats chlich gibt es im Deutschen zwei Methoden eine Aussage zu verneinen
Man kann nicht benutzen und man kann kein benutzen.
Man kann nicht benutzen und man kann kein benutzen.
Und in diesem Video erkl ren wir euch wann man was benutzt
Und in diesem Video erkl ren wir euch wann man was benutzt
und wie man diese s tze formuliert. Los gehtsl
und wie man diese s tze formuliert. Los gehtsl
Der grundlegende Unterschied ist eigentlich einfach
Der grundlegende Unterschied ist eigentlich einfach
Kein benutzt man immer dann wenn man ein Nomen verneint
Kein benutzt man immer dann wenn man ein Nomen verneint
unci zwar ein Nomen mit einem unbestimmten Artikel oder keinem Artikel.
unci zwar ein Nomen mit ei

Wo ich jetzt wohne nicht nee.
Und fr herP h ja. Und was... Wegen zu lauter Musik
Und fr herP h ja. Und was... Wegen zu lauter Musik
N also ber mich noch nicht Gott sei Dank. Bei dir
N also ber mich noch nicht Gott sei Dank. Bei dir
Ja als Kind als Kincl vermutlich fter.
Ja als Kind als Kincl vermutlich fter.
So in der Pubert t habe ich die Musik
So in der Pubert t habe ich die Musik
vielleicht mal zu laut gehabt oder...
vielleicht mal zu laut gehabt oder...
Ich habe auch Musik produziert und bin dabei auch
Ich habe auch Musik produziert und bin dabei auch
manchmal n bisschen zu laut gewesen aber war nie dramatisch.
manchmal n bisschen zu laut gewesen aber war nie dramatisch.
Ja. Ja. Meine Nachbarn haben sich schon ber mich beschwert
Ja. Ja. Meine Nachbarn haben sich schon ber mich beschwert
und sie haben sich Schon ber mich beschwert
und sie haben sich Schon ber mich beschwert
weil ich jetzt immer in meinem Zimmer sport machen muss
weil ich jetzt immer in meinem Zimmer sport machen mus

In [ ]:
import pysrt
subs = pysrt.open(r"D:\Burak\Desktop\Ben.srt")
path = r"D:\Burak\Desktop\FormatFactoryBen.mp4"
cap = cv2.VideoCapture(path)
fps = cap.get(cv2.CAP_PROP_FPS)

for i in range (len(subs)):
    h, m, s, ms = subs[i].start
    t1 = int((h*60*60 + m*60 + s + ms/1000)) 

    h, m, s, ms = subs[i].end
    t2 = int((h*60*60 + m*60 + s + ms/1000))

    text = subs[i].text
    if "<i>" in text:        
        text = text.replace("<i>", " ")
        
    if "</i>" in text:
        text = text.replace("</i>", " ")
    
    xxx = re.findall("[A-Za-z_.]*", text)
        
    while "" in xxx:
        xxx.remove("")
    yyy = ' '.join(xxx)
    text = yyy
    print(str(i+1) +"/"+ str(len(subs)))
    name = "D:\Burak\Desktop\Deneme\\" + text + ".mp4"
    new = VideoFileClip(path).subclip(t1, t2+0.6)
    new.write_videofile(name, threads=4, fps=fps, verbose=False, logger=None)